## Import

In [1]:
# 데이터 분석
import pandas as pd
import numpy as np

In [2]:
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [3]:
# 정규표현식
import re

In [4]:
# 형태소분석 okt
from konlpy.tag import Okt

## 파일 불러오기

In [28]:
df = pd.read_csv('review_data_2t.csv', encoding='utf-8')
df

,review,시설분류
0,?,문화
1,친절해요,문화
2,좋아요 만족스러워요,문화
3,와우~~,문화
4,굿,문화
...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원
141251,친절하십니다,동물병원
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원


## 함수

### 한글만 추출 함수

In [29]:
# 한글만 추출 + 2개이상 공백 1개로 변환
def extract_korean(text):
    # 한글만 추출
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]',' ',text)
    # 공백 2개 이상인 경우 1개로 변환
    text = re.sub(r'\s+',' ',text)
    return text


### 형용사 + 동사 추출 함수

In [30]:
# 형용사 추출하기
def get_adjectives(text):
    okt = Okt() # okt 객체 생성
    okt_pos = okt.pos(text)
    join_text = []
    for text, pos in okt_pos:
        if pos == 'Adjective' or pos == 'Verb':
            join_text.append(text)
    return " ".join(join_text)

## 한글만 추출 (review2 생성)

In [31]:
df['review2'] = df['review'].map(lambda review:extract_korean(review))

In [32]:
df.head(3)

,review,시설분류,review2
0,?,문화,
1,친절해요,문화,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요


## 파생컬럼 생성

In [33]:
# 시간이 오래 걸리는 작업시 시각적으로 진행상황 수치로( %) 보여준다.
from tqdm import tqdm
tqdm.pandas()

In [34]:
df['adj_verb'] = df['review'].progress_map(extract_korean).progress_map(get_adjectives)

100%|█████████████████████████████████████████████████████████████████████████| 141254/141254 [04:28<00:00, 526.78it/s]


In [37]:
df

,review,시설분류,review2,adj_verb
0,?,문화,,
1,친절해요,문화,친절해요,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
3,와우~~,문화,와우,
4,굿,문화,굿,
...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다
141251,친절하십니다,동물병원,친절하십니다,친절하십니다
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요


### 결측치 확인

In [38]:
df.isnull().sum()

review      0
시설분류        0
review2     0
adj_verb    0
dtype: int64

### 데이터의 값이 공백인거 NaN으로 변경

In [39]:
# 데이터값이 공백인거 NaN으로 변경
df_NanTest = df.replace(r'^\s*$', np.nan, regex=True)

In [40]:
df_NanTest

,review,시설분류,review2,adj_verb
0,?,문화,NaN,NaN
1,친절해요,문화,친절해요,친절해요
2,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
3,와우~~,문화,와우,NaN
4,굿,문화,굿,NaN
...,...,...,...,...
141249,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...
141250,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다
141251,친절하십니다,동물병원,친절하십니다,친절하십니다
141252,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요


In [41]:
df_NanTest.isnull().sum()

review          9
시설분류            0
review2      4927
adj_verb    30868
dtype: int64

### review2(한글만 남긴 리뷰)의 컬럼 데이터가 NaN이면 제거

In [42]:
df_NanTest.dropna(subset=['review2'], inplace=True)

In [43]:
df_NanTest.isnull().sum()

review          0
시설분류            0
review2         0
adj_verb    25941
dtype: int64

In [44]:
df_NanTest.dropna(subset=['adj_verb'], inplace=True)

In [45]:
df_NanTest.isnull().sum()

review      0
시설분류        0
review2     0
adj_verb    0
dtype: int64

In [46]:
df_NanTest.shape

(110386, 4)

## 저장

In [47]:
df_NanTest.to_csv('data/review_adj_verb.csv', index=False, encoding='utf-8-sig')

## 감성사전으로 라벨링 작업

### 데이터프레임 불러오기

In [5]:
df_review_data = pd.read_csv('data/review_adj_verb.csv', encoding='utf-8')

In [6]:
df_review_data.head(20)

,review,시설분류,review2,adj_verb
0,친절해요,문화,친절해요,친절해요
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요
2,좋아요,문화,좋아요,좋아요
3,좋아요,문화,좋아요,좋아요
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요
5,산책하기 좋아요,문화,산책하기 좋아요,하기 좋아요
6,처음 가봤는데 산책로가 잘 되어있네요~\n한적하게 산책하며 힐링하기 좋습니다,문화,처음 가봤는데 산책로가 잘 되어있네요 한적하게 산책하며 힐링하기 좋습니다,가봤는데 잘 되어있네요 한적하게 하며 하기 좋습니다
7,보현사에서 시작~구릉산~갈매구릉산자락길~모다아울렛구리.갈매역까지 걷기\n\n갈매역 ...,문화,보현사에서 시작 구릉산 갈매구릉산자락길 모다아울렛구리 갈매역까지 걷기 갈매역 근처에...,있어 하기 좋음 있어 좋음
8,좋습니다,문화,좋습니다,좋습니다
9,좋아요,문화,좋아요,좋아요


### 정규화, 어간 - 추출 함수

In [1]:
# # extract_norm 함수 정의
# def extract_norm(words):
#     if pd.isna(words):
#         return words
    
#     okt = Okt()
#     norm = []
#     for word in words:
#         # 정규화
#         norm.append(okt.normalize(word))
#     return norm

In [2]:
# # extract_stem 함수 정의
# def extract_stem(words):
#     if pd.isna(words):
#         return words
     
#     okt = Okt()
#     stems = []
#     for word in words:
#         # 어간 추출
#         stems.append(okt.morphs(word, stem=True))
#     return stems

In [7]:
def extract_stemming(words, tokenizer):
    # 문자열을 포함한 다양한 타입에서 NaN 확인
    if pd.isna(words):
        return

    # 전체 텍스트에 대해 한 번에 형태소 분석을 수행
    morphs = tokenizer.morphs(words, stem=True)
    
    # 결과에서 어간만 추출하여 조인
    return ' '.join(morph for morph in morphs if morph)

In [8]:
from konlpy.tag import Okt
okt = Okt()

In [9]:
from tqdm import tqdm
tqdm.pandas()
from functools import partial

In [10]:
# extract_stem 함수에 대한 partial 함수 생성
extract_stem_okt = partial(extract_stemming, tokenizer=okt)

In [11]:
# df_review_data['adj_verb_normalized'] = df_review_data['adj_verb'].progress_map(extract_norm)

In [12]:
# df_review_data['adj_verb_stemmed'] = df_review_data['adj_verb'].progress_map(extract_stem)

In [13]:
df_review_data['adj_verb_stemmed'] = df_review_data['adj_verb'].progress_map(extract_stem_okt)

100%|█████████████████████████████████████████████████████████████████████████| 110386/110386 [02:03<00:00, 896.58it/s]


In [14]:
df_review_data

,review,시설분류,review2,adj_verb,adj_verb_stemmed
0,친절해요,문화,친절해요,친절해요,친절하다
1,좋아요 만족스러워요,문화,좋아요 만족스러워요,좋아요 만족스러워요,좋다 만족스럽다
2,좋아요,문화,좋아요,좋아요,좋다
3,좋아요,문화,좋아요,좋아요,좋다
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,휴일날 찾아간 각화제 가을을 부르는 꽃이 피어있네요 ㅋㅋ,찾아간 부르는 피어있네요,찾아가다 부르다 피다
...,...,...,...,...,...
110381,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,동물병원,설사를해서 병원갔어요 배를 만져보고 꼼꼼히 살펴보시더니 약은 안먹어도 되겠다고 하셔...,해서 갔어요 만져 꼼꼼히 살펴보시더니 먹어도 되겠다고 하셔서 맞고 왔는데 다행히 좋...,하다 가다 만지다 꼼꼼하다 살펴보다 먹다 되다 하다 맞다 오다 다행하다 좋다 지다
110382,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립...,동물병원,저희 천사 데려온 곳이고 중성화부터 혈액검사까지 전부 세세하게 봐주셔서 늘 감사드립니다,데려온 세세하게 봐주셔서 드립니다,데려오다 세다 보다 드리다
110383,친절하십니다,동물병원,친절하십니다,친절하십니다,친절하다
110384,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,동물병원,진료 꼼꼼히 잘봐주세요 진료후 간식도 챙겨주시고 선생님께서 산책 가자고하니 요미가 ...,꼼꼼히 봐주세요 챙겨주시고 가자고하니 따라가네요 해주신 약발라주고 있는데 좋아졌어요,꼼꼼하다 보다 챙기다 가다 따라가다 해주다 약바르다 있다 좋아지다


### 감성사전 불러오기

In [15]:
import json

In [16]:
with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    Sentiword_info = json.load(f)

In [17]:
sentiword_dic = pd.DataFrame(Sentiword_info)

In [18]:
sentiword_dic

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


#### 감성사전 추가하기

In [19]:
# 추가할 단어 있으면 추가하기
new_row = {'word': '친절해요', 'word_root': '친절', 'polarity' : '2'}
sentiword_dic = pd.concat([sentiword_dic, pd.DataFrame(new_row, index=[0])], ignore_index=True)

### 라벨링 작업

In [ ]:
# df = pd.DataFrame(columns=('adj_verb', 'sentiment'))
# idx = 0 # 다음 리뷰로 넘기기 위한 초기값

# for adj_verb in df_review_data['adj_verb']:  # df_review_data의 adj_verb 열의 각 토큰에 대해 루프
#     sentiment = 0  # 초기 감성값 0으로 설정
#     for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
#         if sentiword_dic.word_root[i] in adj_verb and sentiword_dic.word[i] in adj_verb:  # 리뷰 문장에 감성 단어가 있는지 확인
#             sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
#     df.loc[idx] = [adj_verb, sentiment]  # 리뷰별 극성값을 데이터프레임으로 쌓음
#     idx += 1  # 다음 리뷰 문장으로 넘어감


In [84]:
df = pd.DataFrame(columns=('review', '시설분류', 'adj_verb_stemmed', 'sentiment', 'label'))
idx = 0 # 다음 리뷰로 넘기기 위한 초기값

for idx, row in df_review_data.iloc[:20].iterrows():  # 처음 20개의 행에 대해 루프
    review = row['review']  # 리뷰 가져오기
    시설분류 = row['시설분류']  # 시설분류 가져오기
    adj_verb_stemmed = row['adj_verb_stemmed']  # adj_verb 가져오기
    sentiment = 0  # 초기 감성값 0으로 설정
    for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
        if sentiword_dic.word_root[i] in adj_verb_stemmed and sentiword_dic.word[i] in adj_verb_stemmed:  # 리뷰 문장에 감성 단어가 있는지 확인
            sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
    label = 1 if sentiment >= 0 else 0  # sentiment 값에 따라 label 값 할당
    df.loc[idx] = [review, 시설분류, adj_verb_stemmed, sentiment, label]  # 리뷰별 극성값과 label을 데이터프레임으로 쌓음


In [85]:
df

,review,시설분류,adj_verb_stemmed,sentiment,label
0,친절해요,문화,친절하다,4,1
1,좋아요 만족스러워요,문화,좋다 만족스럽다,6,1
2,좋아요,문화,좋다,2,1
3,좋아요,문화,좋다,2,1
4,휴일날 찾아간 각화제~\n가을을 부르는 꽃이 피어있네요 ㅋㅋ,문화,찾아가다 부르다 피다,0,1
5,산책하기 좋아요,문화,하다 좋다,2,1
6,처음 가봤는데 산책로가 잘 되어있네요~\n한적하게 산책하며 힐링하기 좋습니다,문화,가보다 자다 되어다 한적하다 하다 하다 좋다,2,1
7,보현사에서 시작~구릉산~갈매구릉산자락길~모다아울렛구리.갈매역까지 걷기\n\n갈매역 ...,문화,있다 하다 좋다 있다 좋다,2,1
8,좋습니다,문화,좋다,2,1
9,좋아요,문화,좋다,2,1


In [89]:
# df = pd.DataFrame(columns=('review', '시설분류', 'adj_verb_stemmed', 'sentiment', 'label'))
# idx = 0 # 다음 리뷰로 넘기기 위한 초기값

# for idx, row in df_review_data.iterrows():  # 처음 20개의 행에 대해 루프
#     review = row['review']  # 리뷰 가져오기
#     시설분류 = row['시설분류']  # 시설분류 가져오기
#     adj_verb = row['adj_verb_stemmed']  # adj_verb 가져오기
#     sentiment = 0  # 초기 감성값 0으로 설정
#     for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
#         if sentiword_dic.word_root[i] in adj_verb and sentiword_dic.word[i] in adj_verb:  # 리뷰 문장에 감성 단어가 있는지 확인
#             sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
#     label = 1 if sentiment >= 0 else 0  # sentiment 값에 따라 label 값 할당
#     df.loc[idx] = [review, 시설분류, adj_verb, sentiment, label]  # 리뷰별 극성값과 label을 데이터프레임으로 쌓음


In [ ]:
df = pd.DataFrame(columns=('review', '시설분류', 'adj_verb_stemmed', 'sentiment', 'label'))
idx = 0 # 다음 리뷰로 넘기기 위한 초기값

for idx, row in tqdm(df_review_data.iterrows(), total=len(df_review_data)): 
    review = row['review']  # 리뷰 가져오기
    시설분류 = row['시설분류']  # 시설분류 가져오기
    adj_verb_stemmed = row['adj_verb_stemmed']  # adj_verb 가져오기
    sentiment = 0  # 초기 감성값 0으로 설정
    for i in range(len(sentiword_dic)):  # 감성사전의 모든 단어에 대해 루프
        if sentiword_dic.word_root[i] in adj_verb_stemmed and sentiword_dic.word[i] in adj_verb_stemmed:  # 리뷰 문장에 감성 단어가 있는지 확인
            sentiment += int(sentiword_dic.polarity[i])  # 감성단어가 있다면 극성값 합계를 구함
    label = 1 if sentiment >= 0 else 0  # sentiment 값에 따라 label 값 할당
    df.loc[idx] = [review, 시설분류, adj_verb_stemmed, sentiment, label]  # 리뷰별 극성값과 label을 데이터프레임으로 쌓음


 15%|███████████▎                                                             | 17045/110386 [33:49<3:34:53,  7.24it/s]

In [85]:
df

,review,시설분류,adj_verb,sentiment,label
0,"[[친], [절], [해], [요]]",문화,친절해요,0,1
1,"[[좋다], [아], [요], [], [만], [족], [스], [러], [워], ...",문화,좋아요 만족스러워요,4,1
2,"[[좋다], [아], [요]]",문화,좋아요,2,1
3,"[[좋다], [아], [요]]",문화,좋아요,2,1
4,"[[찾다], [아], [간], [], [부], [르], [늘다], [], [피], ...",문화,찾아간 부르는 피어있네요,0,1
5,"[[하], [기], [], [좋다], [아], [요]]",문화,하기 좋아요,2,1
6,"[[가다], [보다], [늘다], [데], [], [자다], [], [되다], [어...",문화,가봤는데 잘 되어있네요 한적하게 하며 하기 좋습니다,0,1
7,"[[있다], [어], [], [하], [기], [], [좋다], [음], [], [...",문화,있어 하기 좋음 있어 좋음,2,1
8,"[[좋다], [습], [니], [다]]",문화,좋습니다,0,1
9,"[[좋다], [아], [요]]",문화,좋아요,2,1


In [25]:
from tqdm import tqdm

# tqdm에 대한 pandas의 apply 함수를 활성화합니다.
tqdm.pandas()

In [26]:
# 감정 점수 계산 함수
def calculate_sentiment(row):
    sentiment = 0
    for _, dic_row in sentiword_dic.iterrows():
        if dic_row['word_root'] == row['adj_verb_stemmed'] and dic_row['word'] == row['adj_verb_stemmed']:
            sentiment += dic_row['polarity']
            break
    return sentiment

In [27]:
# 감정 점수 및 레이블 계산 (tqdm을 사용하여 진행률을 표시합니다.)
df_review_data['sentiment'] = df_review_data.progress_apply(calculate_sentiment, axis=1)
df_review_data['label'] = df_review_data['sentiment'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))

# 최종 DataFrame 준비
df = df_review_data[['review', 'facility_type', 'adj_verb_stemmed', 'sentiment', 'label']]

  4%|██▌                                                                      | 3968/110386 [53:21<23:51:09,  1.24it/s]


KeyboardInterrupt: 

In [ ]:
df.to_csv('data/knu_data_review_test2.csv', index=False, encoding='utf-8-sig')